In [1]:
import pandas as pd
import numpy as np
import os
from hsmm_core.data_utils import TradingHours, DataLoader
from hsmm_core.labelling import DataLabellingSimple
from hsmm_core.consts import ThresholdMethod, LabellingChoice
import datetime as dt
from pandas.tseries.offsets import BDay
from hsmm_core.utils import mc_limiting_distribution, states_from_limit_dist
from hsmm_core.observation_models import ExpIndMixDiracGauss
from hsmm_core.feature_spaces import hmm_features
from hsmm_core.hsmm_runner import HmmCalibration

from hsmm_core.hmm import hmm_impl

from hsmm_core.data_utils import DataLoader, TradingHours
from hsmm_core.labelling import DataLabellingSimple
from hsmm_core.consts import ThresholdMethod, LabellingChoice
import pickle
from hsmm_core.consts import InitialisationMethod
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,  GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
sc = StandardScaler()


In [2]:
# ticker = 'SYNT_2states' #testing a new synthetic ticker


ticker = 'VOD.L'

data_dir = os.getenv('FINANCE_DATA') #main directory
features_path = os.path.join(os.path.expanduser("~"), 'FinData/features_models/features/') #where features are saved
labels_path = os.path.join(os.path.expanduser("~"), 'FinData/features_models/labels') #where labels are saved
ticker_labels_path = os.path.join(labels_path, ticker + '/NON_DIRECTIONAL')

if not os.path.exists(os.path.join(data_dir, ticker)):
    os.makedirs(os.path.join(data_dir, ticker))

if not os.path.exists(ticker_labels_path):
    os.makedirs(ticker_labels_path)

labels_list = os.listdir(ticker_labels_path)

# ####paths####
main_path = os.path.join(os.path.expanduser("~"), 'Data/features_models/')

models_path = os.path.join(main_path, 'models')
ticker_models_path = os.path.join(models_path, ticker)

hmm_models_path = os.path.join(models_path,'hmm_models') #only if we store the hmm models
if not os.path.exists(ticker_models_path):
    os.makedirs(ticker_models_path)


In [3]:
os.path.join(os.path.expanduser("~"), 'FinData')

'/home/ak/FinData'

In [4]:
data_dir_list= os.listdir(data_dir)

user_dir=os.listdir(os.path.join(os.path.expanduser("~"), 'FinData'))
ftse_list= [s for s in user_dir if s.endswith('.L')]
tmp ='/home/ak/tmp/'
ftse_list

['SMIN.L',
 'CEY.L',
 'RR.L',
 'RTO.L',
 'TSCO.L',
 'ULVR.L',
 'CPI.L',
 'LAND.L',
 'WPP.L',
 'UU.L',
 'MKS.L',
 'BLT.L',
 'AV.L',
 'VOD.L',
 'RDSa.L',
 'III.L',
 'RDSb.L',
 'ECM.L',
 'PSON.L',
 'REL.L',
 'IEER.L',
 'BARC.L',
 'GKN.L',
 'NG.L',
 'RB.L',
 'RSA.L',
 'ITV.L',
 'AAL.L',
 'IOG.L',
 'SDR.L',
 'STAN.L',
 'MAB.L',
 'LGEN.L',
 'RBS.L',
 'LLOY.L',
 'CPG.L',
 'HSBA.L',
 'EGS.L',
 'AZN.L',
 'SGE.L',
 'DMGOa.L',
 'APF.L',
 'PRU.L',
 'CNA.L',
 'SHP.L',
 'DGE.L',
 'BATS.L',
 'KGF.L',
 'CCL.L',
 'SPT.L']

In [5]:
ticker = ftse_list[0]

# sd = '20170116'
# ed = '20170120'


trading_hours_filter = TradingHours.only_mkt_hours

data_loader = DataLoader(trading_hours_filter, ccy_bar= 1000)

data = data_loader.load_trades_data(ticker)#, start_date=sd, end_date=ed)


labelling_method_params = [
# {
#     'labelling_method': LabellingChoice.ep_in_window,
#     'rolling_window': 10,
#     'threshold_method': ThresholdMethod.historical_vol_ma,
#     'threshold_vol_window': 5,
#     'updown_thrshd_vol_pct': 10.,
#     'trading_hours_filter': trading_hours_filter,
# },
{
    'labelling_method': LabellingChoice.price_move_in_window,
    'rolling_window': 2,
    # Uncomment below if you want to check a price move only above a certain level
    # 'updown_threshold': 0.1
    # 'threshold_method': ThresholdMethod.arbitrary,
}]

for label_init in labelling_method_params:
    labeller = DataLabellingSimple(label_init)
    the_label = labeller.get_label_name()
    labeller.label_training_data(data)

print "ok"


ok


In [6]:
window = 25
threshold = 0.1

labelling_method_params = [{

    'labelling_method': LabellingChoice.price_move_in_window,
    'rolling_window': window,
    # Uncomment below if you want to check a price move only above a certain level
    'updown_threshold': threshold,  # this is multiplied by 100
    'threshold_method': ThresholdMethod.arbitrary,
}]

for label_init in labelling_method_params:
    print label_init
    labeller = DataLabellingSimple(label_init)
    labeller.label_training_data(data)
#
#     # # TODO IS THIS WHAT YOU WANTED TO DO ? SAVE THE DATA WITH LABELS OR JUST THE LABELS ?
for date, date_data in data.iteritems():
    date_data.to_csv(os.path.join(tmp, str(date)+'.csv'))

{'rolling_window': 25, 'labelling_method': 'PrMov', 'updown_threshold': 0.1, 'threshold_method': 'arbitrary'}


In [8]:
data.keys()

['20170731',
 '20170831',
 '20170830',
 '20170727',
 '20170726',
 '20170721',
 '20170720',
 '20170929',
 '20170928',
 '20170725',
 '20170724',
 '20170925',
 '20170927',
 '20170926',
 '20170921',
 '20170920',
 '20170922',
 '20170808',
 '20170809',
 '20170807',
 '20170804',
 '20170802',
 '20170803',
 '20170801',
 '20170728',
 '20170717',
 '20170714',
 '20170712',
 '20170713',
 '20170710',
 '20170711',
 '20170718',
 '20170719',
 '20170811',
 '20170810',
 '20170815',
 '20170814',
 '20170817',
 '20170816',
 '20170818',
 '20170703',
 '20170705',
 '20170704',
 '20170707',
 '20170706',
 '20180328',
 '20180329',
 '20180326',
 '20180327',
 '20180320',
 '20180321',
 '20180322',
 '20180323',
 '20180419',
 '20180410',
 '20180411',
 '20180420',
 '20180302',
 '20180418',
 '20180301',
 '20180306',
 '20180307',
 '20180305',
 '20180412',
 '20180413',
 '20180308',
 '20180309',
 '20180416',
 '20180417',
 '20170829',
 '20170914',
 '20170915',
 '20170911',
 '20170912',
 '20170913',
 '20170918',
 '20170919',

In [ ]:
    labelling_method_params = [{
    
            'labelling_method': LabellingChoice.price_move_in_window,
            'rolling_window': window,
            # Uncomment below if you want to check a price move only above a certain level
            'updown_threshold': threshold,  # this is multiplied by 100
            'threshold_method': ThresholdMethod.arbitrary,
        }]
    
        for label_init in labelling_method_params:
            print label_init
            labeller = DataLabellingSimple(label_init)
            labeller.label_training_data(data)

In [9]:
data_keys=data.keys()
# data_vol_clock[rel_data_keys[1]]

In [ ]:
os.listdir(features_path)

In [ ]:
##get dates as a list 
data_files=os.listdir(os.path.join(data_dir, ticker))

synt_dates=[os.path.splitext(x)[0] for x in data_files]

In [ ]:
synt_dates.sort()
start_date=synt_dates[0]
end_date =synt_dates[-1]
start_date, end_date

In [9]:
data_loader_init = {
        'trading_hours_filter': TradingHours.only_mkt_hours
    }

no_states=2

hmm_init = {
    'obs_model_name': 'CensoredExpIndMixDiracGauss',
    'em_obs_init_method': InitialisationMethod.cluster,
    'em_hidden_init_method': InitialisationMethod.uniform,
    'no_hidden_states': no_states,
    'update_tag': 'tpsml'
}

data_loader = DataLoader(**data_loader_init)
# keep the hash of the data loader to uniquely identify how the data was loaded ( perhaps a dollar clock was
# used), as this affects the calibration of the hmm
data_loader_hash = data_loader.data_loader_hash()

data = data_loader.load_trades_data(ticker)#, start_date=start_date, end_date=end_date)
##calibrate the hmm models
hmm_calibration_engine = HmmCalibration(init_params=hmm_init)
hmm_calibration_engine.run_calibration_all_data(ticker, data, data_loader_hash,
                                                force_recalc=False, use_multiprocessing=False,
                                                n_processes=2)

# Create the hmm feature engine and for every change the hmm model in the features engine
features_engine = hmm_features()

Number of points in data set is 1054, number of points with large price change 246
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 970, number of points with large price change 242
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 799, number of points with large price change 231
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 1067, number of points with large price change 331
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 893, number of points with large price change 251
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 1506, number of points with large price change 391
Clustering of data produces highly unbalanced subsets; 

Number of points in data set is 10622, number of points with large price change 5674
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 1735, number of points with large price change 772
Number of points in data set is 2109, number of points with large price change 587
Number of points in data set is 1683, number of points with large price change 619
Number of points in data set is 2151, number of points with large price change 810
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 1843, number of points with large price change 830
Number of points in data set is 1993, number of points with large price change 765
Clustering of data produces highly unbalanced subsets; fall back to random intialisation.
Number of points in data set is 1752, number of points with large price change 745
Clustering of data produces highly unbalanced subsets; fall back

In [11]:
features_engine.em_features

<bound method hmm_features.em_features of <hsmm_core.feature_spaces.hmm_features object at 0x7fb507e22490>>

In [ ]:
data_loader_hash

In [12]:
tmp='/home/ak/tmp/'
import pickle
import datetime as dt


In [13]:
now=dt.datetime.now()
print(now.strftime('%Y%m%d'))

20181223


In [55]:
for date, date_data in data.iteritems():
    print(date)
    stored_hmm, _ = hmm_calibration_engine.get_calibrated_hmm(ticker, date, data_loader_hash)
    stored_hmm_file = "_".join(
                (str(ticker),"model_date_:", str(date), str(no_states), "states", "stored_hmm", now.strftime('%Y%m%d'), ".pickle"))
    pickle.dump(stored_hmm, open(os.path.join(tmp,stored_hmm_file), 'wb')) #storing the hmm model
    features_engine.hmm = stored_hmm
    list_fwd_dates=fwd_dates(dates_list=list(data.keys()), key_date=date)
    for fwd_date in list_fwd_dates:
        features_load = features_engine.generate_features(data[fwd_date])
        features_file = "_".join((str(ticker),str(fwd_date),str(no_states),"states","features_date",now.strftime('%Y%m%d'),".pickle"))
        print(features_file)
        pickle.dump(features_load, open(os.path.join(tmp, features_file), 'wb'))

20170731
SMIN.L_20170831_2_states_features_date_20181223_.pickle
SMIN.L_20170830_2_states_features_date_20181223_.pickle
SMIN.L_20170929_2_states_features_date_20181223_.pickle
SMIN.L_20170928_2_states_features_date_20181223_.pickle
SMIN.L_20170925_2_states_features_date_20181223_.pickle
SMIN.L_20170927_2_states_features_date_20181223_.pickle
SMIN.L_20170926_2_states_features_date_20181223_.pickle
SMIN.L_20170921_2_states_features_date_20181223_.pickle
SMIN.L_20170920_2_states_features_date_20181223_.pickle
SMIN.L_20170922_2_states_features_date_20181223_.pickle
SMIN.L_20170808_2_states_features_date_20181223_.pickle
SMIN.L_20170809_2_states_features_date_20181223_.pickle
SMIN.L_20170807_2_states_features_date_20181223_.pickle
SMIN.L_20170804_2_states_features_date_20181223_.pickle
SMIN.L_20170802_2_states_features_date_20181223_.pickle
SMIN.L_20170803_2_states_features_date_20181223_.pickle
SMIN.L_20170801_2_states_features_date_20181223_.pickle
SMIN.L_20170811_2_states_features_date_

KeyboardInterrupt: 

In [36]:

def fwd_dates(dates_list, key_date):
    # returns a list of dates that are forward from the key_date
    fwd_dates_list = [i for i in dates_list if i > key_date]
    return fwd_dates_list

In [45]:
# list(data.keys())
# for date, date_data in data.iteritems():
#     print len(fwd_dates(dates_list=list(data.keys()), key_date=date))
date_data

,TradedTime,TradedPrice,ReturnTradedPrice,Volume,Duration
0,08:00:36.274658,15.620000,0.000000,100.0,0.000000
1,08:02:22.947225,15.630000,0.000640,244.0,106.672567
2,08:02:23.047689,15.630000,0.000000,98.0,0.100464
3,08:02:26.937219,15.630000,0.000000,39.0,3.889530
4,08:03:07.726214,15.620000,-0.000640,482.0,40.788995
5,08:03:07.970289,15.620000,0.000000,238.0,0.244075
6,08:03:07.970300,15.620000,0.000000,52.0,0.000011
7,08:04:12.873657,15.620000,0.000000,310.0,64.903357
8,08:04:12.873669,15.620000,0.000000,160.0,0.000012
9,08:04:12.873681,15.620000,0.000000,176.0,0.000012


In [ ]:
data= features_load
file_name= os.path.join(tmp,'data_tmp.pickle')
with open(file_name, 'wb') as f:
    pickle.dump(data, f)

In [ ]:
with open(file_name, 'rb') as f:
     data_read = pickle.load(f)

In [ ]:
data_read

#     # This is sequence for the name of the best classifiers.
    #     seq_clf = "_".join(("synt_model",  str(date), labels_clean.columns.values[0], "clfs", ".pickle"))
    #     print("saving the classifiers:", seq_clf)
    #     pickle.dump(best_clfs, open(os.path.join(ticker_models_path, seq_clf), 'wb'))
    #

In [ ]:
# This is sequence for the name of the best classifiers.
    #     seq_clf = "_".join(("synt_model",  str(date), labels_clean.columns.values[0], "clfs", ".pickle"))
    #     print("saving the classifiers:", seq_clf)
    #     pickle.dump(best_clfs, open(os.path.join(ticker_models_path, seq_clf), 'wb'))
    #a bit co

## create labels and store the labels with the data ###

In [ ]:
window = 25
threshold = 0.1

labelling_method_params = [{

    'labelling_method': LabellingChoice.price_move_in_window,
    'rolling_window': window,
    # Uncomment below if you want to check a price move only above a certain level
    'updown_threshold': threshold,  # this is multiplied by 100
    'threshold_method': ThresholdMethod.arbitrary,
}]

for label_init in labelling_method_params:
    print label_init
    labeller = DataLabellingSimple(label_init)
    labeller.label_training_data(data)

# SAVE THE DATA WITH LABELS 
for date, date_data in data.iteritems():
    date_data.to_csv(os.path.join(ticker_labels_path, str(date)+'.csv'))

In [ ]:
dd_keys=data.keys()


## create features for out of sample predictions ##